In [5]:
N = 10_000_000
x = list(range(N))

In [6]:
import random

random.shuffle(x)

In [8]:
import numpy as np

In [11]:
np.random.shuffle(x)

In [12]:
x[0]

6425110

In [1]:
import sys
sys.path.append("..")

In [2]:
import json
import logging
import os
import math
import shutil
from pathlib import Path
from itertools import chain

# from dotenv import load_dotenv
import torch
import numpy as np
import datasets
import transformers
from torch.utils.data import DataLoader
from huggingface_hub import hf_hub_download

from lm_experiments_tools.trainer_accelerate import TrainerAccelerate as Trainer, TrainerAccelerateArgs as TrainerArgs

from torch.nn.utils.rnn import pad_sequence

import accelerate

# load_dotenv()

logger_fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
logging.basicConfig(format=logger_fmt, level=logging.INFO)
logger = logging.getLogger('')


# if CUDA_VISIBLE_DEVICES is not set make all gpus visible
if os.environ.get('CUDA_VISIBLE_DEVICES', None) is None:
    os.environ['CUDA_VISIBLE_DEVICES'] = ','.join([str(i) for i in range(torch.cuda.device_count())])

logger.info(f"CUDA_VISIBLE_DEVICES: {os.environ['CUDA_VISIBLE_DEVICES']}")
# first call to torch.cuda.device_count() sets visible gpus, following calls will not change the result
logger.info(f"CUDA DEVICE COUNT: {torch.cuda.device_count()}")

# import transformers  # noqa: E402
from transformers import AutoConfig, AutoTokenizer, HfArgumentParser  # noqa: E402

from lm_experiments_tools.utils import get_cls_by_name, get_optimizer, prepare_run  # noqa: E402
import lm_experiments_tools.optimizers as optimizers  # noqa: E402


libgomp: Invalid value for environment variable OMP_NUM_THREADS
/home/jovyan/envs/accel_rmt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-09-04 12:53:47,908] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-09-04 12:53:49,086 - root - INFO - CUDA_VISIBLE_DEVICES: 0,1,2,3,4,5,6,7
2023-09-04 12:53:49,088 - root - INFO - CUDA DEVICE COUNT: 8


In [3]:
class Test(torch.nn.Module):
    pass

In [4]:
model = Test()

In [5]:
isinstance(model, torch.nn.Module)

True

In [3]:

NUM_SYMBOLS = 16
def generate_pairs(key_size, value_size, num_pairs):
    keys = torch.randint(0, NUM_SYMBOLS, (num_pairs * 2, key_size))
    keys[:, 0] = torch.randint(1, NUM_SYMBOLS, (num_pairs * 2, ))
    
    unique = keys.unique(dim=0)
    delta_pairs = num_pairs - unique.shape[0]
    if delta_pairs > 0:
        print('got unique')
        return generate_pairs(key_size, value_size, num_pairs)

    selected_ids = torch.randperm(unique.shape[0])[:num_pairs]
    keys = unique[selected_ids]

    values = torch.randint(0, NUM_SYMBOLS, (num_pairs, value_size))
    values[:, 0] = torch.randint(1, NUM_SYMBOLS, (num_pairs, ))
    return keys, values


class ARDataset:
    def __init__(self, key_size, value_size, sample_len=1, num_samples=20_000):
        self.sample_len = sample_len
        keys, values = generate_pairs(key_size, value_size, sample_len * num_samples)

        self.keys = keys.reshape(num_samples, -1)
        self.values = values.reshape(num_samples, -1)
        self.target_key_inds = torch.randint(sample_len, (num_samples, ))

    def __getitem__(self, idx):
        keys, values, tgt_ind = self.keys[idx], self.values[idx], self.target_key_inds[idx]
        dim = 0 if keys.ndim == 1 else 1
        keys = torch.chunk(keys, self.sample_len, dim=dim)
        values = torch.chunk(values, self.sample_len, dim=dim)

        sample = {'keys': keys, 'values': values, 'target_key_ind': tgt_ind}
        return sample
    
    def __len__(self):
        return self.keys.shape[0]

In [4]:
class Holder:
    def __init__(self):
        pass
    def __getattr__(self, name):
        try:
            return super().__getattr__(name)
        except AttributeError:
            return None

args = Holder()

In [5]:
# args.from_pretrained = 'gpt2'

In [107]:

### 1seg 

attrs = ("--model_path", "/cephfs/home/bulatov/bulatov/RMT_light/runs/test/cnli_original",
"--task_name", "exact_match",
"--model_type", "decoder",
"--memory_cell_cls", "modeling_rmt.experimental:MemoryCellGenerate",
"--recurrent_wrapper_cls", "modeling_rmt.language_modeling:RecurrentWrapper",
"--model_cls", "base_models.modeling_gpt_neox:GPTNeoXForCausalLM", 
"--model_cfg",  "/home/jovyan/rmt/wip/base_models/gptconfigs/neox_tiny.json",
"--model_cpt",  "/home/jovyan/rmt/runs/associative_retrieval/lr1e-03_linear_adamw_wd1e-03_k4-v16-p1-1x2048_mem4_bs512_regular_bptt-1/run_4/",
"--dataset_path", "/home/jovyan/rmt/datasets/associative_retrieval/",
"--optimizer", "AdamW",
"--weight_decay", "0.001",
"--lr", "1e-03", 
"--lr_scheduler", "constant_with_warmup",
"--clip_grad_value", "5.0")

int_attrs = (
    # "--input_seq_len", "512",
"--input_size", "2048",
"--key_size", "4",
"--value_size", "16",
"--num_pairs", "1",
"--train_size", "10000",
"--valid_size", "1000",
"--test_size", "1000",
"--segment_size", "44",
"--num_mem_tokens", "4",
"--max_n_segments" ,"1", 
"--batch_size", "2", 
"--gradient_accumulation_steps", "1",
# "--use_generate_on_valid",
"--iters", "100",
"--num_warmup_steps", "100",
"--data_n_workers", "2",
"--log_interval", "10",
"--show_valid_examples", "5",
"--early_stopping_patience", "15",
"--seed", "42",
"--k2", "-1",)

In [106]:
# ### 2seg 

# attrs = ("--model_path", "/cephfs/home/bulatov/bulatov/RMT_light/runs/test/cnli_original",
# "--task_name", "exact_match",
# "--model_type", "decoder",
# "--memory_cell_cls", "modeling_rmt.experimental:MemoryCellGenerate",
# "--recurrent_wrapper_cls", "modeling_rmt.language_modeling:RecurrentWrapper",
# "--model_cls", "base_models.modeling_gpt_neox:GPTNeoXForCausalLM", 
# "--model_cfg",  "/home/jovyan/rmt/wip/base_models/gptconfigs/neox_tiny.json",
# "--model_cpt",  "/home/jovyan/rmt/runs/associative_retrieval/lr1e-03_linear_adamw_wd1e-03_k4-v8-p1-2x2048_mem4_bs512_regular_bptt-2_from_cpt/run_4/",
# "--dataset_path", "/home/jovyan/rmt/datasets/associative_retrieval/",
# "--optimizer", "AdamW",
# "--weight_decay", "0.001",
# "--lr", "1e-03", 
# "--lr_scheduler", "constant_with_warmup",
# "--clip_grad_value", "5.0")

# int_attrs = (
#     # "--input_seq_len", "512",
# "--input_size", "2048",
# "--key_size", "4",
# "--value_size", "8",
# "--num_pairs", "1",
# "--train_size", "10000",
# "--valid_size", "1000",
# "--test_size", "1000",
# "--segment_size", "14",
# "--num_mem_tokens", "4",
# "--max_n_segments" ,"2", 
# "--batch_size", "2", 
# "--gradient_accumulation_steps", "1",
# # "--use_generate_on_valid",
# "--iters", "100",
# "--num_warmup_steps", "100",
# "--data_n_workers", "2",
# "--log_interval", "10",
# "--show_valid_examples", "5",
# "--early_stopping_patience", "15",
# "--seed", "42",
# "--k2", "-1",)


In [94]:
# ### 1-2seg 

# attrs = ("--model_path", "/cephfs/home/bulatov/bulatov/RMT_light/runs/test/cnli_original",
# "--task_name", "exact_match",
# "--model_type", "decoder",
# "--memory_cell_cls", "modeling_rmt.experimental:MemoryCellGenerate",
# "--recurrent_wrapper_cls", "modeling_rmt.language_modeling:RecurrentWrapper",
# "--model_cls", "base_models.modeling_gpt_neox:GPTNeoXForCausalLM", 
# "--model_cfg",  "/home/jovyan/rmt/wip/base_models/gptconfigs/neox_tiny.json",
# "--model_cpt",  "/home/jovyan/rmt/runs/associative_retrieval/lr1e-03_linear_adamw_wd1e-03_k4-v16-p1-1x2048_mem4_bs512_regular_bptt-1/run_4/",
# "--dataset_path", "/home/jovyan/rmt/datasets/associative_retrieval/",
# "--optimizer", "AdamW",
# "--weight_decay", "0.001",
# "--lr", "1e-03", 
# "--lr_scheduler", "constant_with_warmup",
# "--clip_grad_value", "5.0")

# int_attrs = (
#     # "--input_seq_len", "512",
# "--input_size", "2048",
# "--key_size", "4",
# "--value_size", "16",
# "--num_pairs", "1",
# "--train_size", "10000",
# "--valid_size", "1000",
# "--test_size", "1000",
# "--segment_size", "22",
# "--num_mem_tokens", "4",
# "--max_n_segments" ,"2", 
# "--batch_size", "2", 
# "--gradient_accumulation_steps", "1",
# # "--use_generate_on_valid",
# "--iters", "100",
# "--num_warmup_steps", "100",
# "--data_n_workers", "2",
# "--log_interval", "10",
# "--show_valid_examples", "5",
# "--early_stopping_patience", "15",
# "--seed", "42",
# "--k2", "-1",)


In [108]:
for a, v in zip(int_attrs[::2], int_attrs[1::2]):
    setattr(args, a.split('--')[1], int(v))

for a, v in zip(attrs[::2], attrs[1::2]):
    try:
        setattr(args, a.split('--')[1], float(v))
    except ValueError:
        setattr(args, a.split('--')[1], v)

In [109]:
block_size = args.segment_size
sep_token, gen_token, eos_token = 100, 101, 102

def collate_fn(batch):
    keys = [b['keys'] for b in batch]
    values = [b['values'] for b in batch]
    tgt_inds = [b['target_key_ind'].item() for b in batch]

    bs = len(keys)
    sep_tokens = torch.ones(bs, 1) * sep_token
    eos_tokens = torch.ones(bs, 1) * eos_token
    gen_tokens = torch.ones(bs, 1) * gen_token
    sample = []
    for i in range(args.num_pairs):
        sample.append(torch.stack([k[i] for k in keys]))
        sample.append(sep_tokens)
        sample.append(torch.stack([v[i] for v in values]))
        sample.append(eos_tokens)

    target_keys = torch.stack([k[i] for i, k in zip(tgt_inds, keys)])
    target_values = torch.stack([k[i] for i, k in zip(tgt_inds, values)])

    sample.append(target_keys)
    sample.append(gen_tokens)

    input_ids_generate = torch.cat(sample, dim=1)

    sample.append(target_values)
    sample.append(eos_tokens)
    input_ids = torch.cat(sample, dim=1)

    labels_mask = torch.zeros_like(input_ids).bool()
    labels_mask[:, -args.value_size - 2:] = True

    collated = {'input_ids': input_ids.long(), 
                'input_ids_generate': input_ids_generate.long(), 
                'attention_mask': torch.ones_like(input_ids).bool(),
                'attention_mask_generate': torch.ones_like(input_ids_generate).bool(),
                'labels': input_ids.long(), 
                'labels_mask': labels_mask, 
                }
    return collated

In [110]:
dataset_name = f"AR_k{args.key_size}_v{args.value_size}_p{args.num_pairs}"
path = os.path.join(args.dataset_path, dataset_name)

# if os.path.exists(path):
#     print(f"Loading {dataset_name} from disk.")
#     train_dataset = torch.load(os.path.join(path, 'train'))
#     valid_dataset = torch.load(os.path.join(path, 'valid'))
#     test_dataset = torch.load(os.path.join(path, 'test'))
# else:
# os.system(f"mkdir {path}")
train_dataset = ARDataset(args.key_size, args.value_size, sample_len=args.num_pairs, num_samples=args.train_size)
valid_dataset = ARDataset(args.key_size, args.value_size, sample_len=args.num_pairs, num_samples=args.valid_size)
test_dataset = ARDataset(args.key_size, args.value_size, sample_len=args.num_pairs, num_samples=args.test_size)

# torch.save(train_dataset, os.path.join(path, 'train'))
# torch.save(valid_dataset, os.path.join(path, 'valid'))
# torch.save(test_dataset,  os.path.join(path, 'test'))

In [111]:
per_worker_batch_size = 2 # args.batch_size * args.gradient_accumulation_steps
kwargs = {'pin_memory': True, 'num_workers': args.data_n_workers}
train_dataloader = DataLoader(train_dataset, batch_size=per_worker_batch_size,
                                collate_fn=collate_fn, **kwargs)
valid_dataloader = DataLoader(valid_dataset, batch_size=per_worker_batch_size,
                                collate_fn=collate_fn, **kwargs)
test_dataloader = DataLoader(test_dataset, batch_size=per_worker_batch_size,
                                collate_fn=collate_fn, **kwargs)

In [112]:
model_cls = get_cls_by_name(args.model_cls)


In [113]:
if not args.from_pretrained:
    model_cfg = AutoConfig.from_pretrained(args.model_cfg)
    model = model_cls(config=model_cfg)
else:
    logger.info(f'Loading pretrained model: {args.from_pretrained}')
    model = model_cls.from_pretrained(args.from_pretrained)

# ## add [GEN] token
# model.resize_token_embeddings(len(tokenizer))

## load cpt of backbone model
if args.backbone_cpt:
    backbone_cpt = os.path.join(args.backbone_cpt, "model_best.pth")
    cpt = torch.load(backbone_cpt, map_location='cpu')
    model.load_state_dict(cpt['model_state_dict'], strict=False)
    logger.info(f'Loaded baseline state dict from: {args.backbone_cpt}')

# Pass memory settings to pretrained model
if args.num_mem_tokens is not None:
    memory_cell_cls = get_cls_by_name(args.memory_cell_cls)
    recurrent_wrapper_cls = get_cls_by_name(args.recurrent_wrapper_cls)
    logger.info(f'Wrapping in: {memory_cell_cls} and {recurrent_wrapper_cls}')
    
    
    cell = memory_cell_cls(model, args.num_mem_tokens)
    model = recurrent_wrapper_cls(cell, 
                                    segment_size=block_size,
                                    max_n_segments=args.max_n_segments, 
                                #   vary_n_segments=args.vary_n_segments,
                                    k2=args.k2,
                                    segment_alignment=args.segment_alignment
    )

2023-08-31 13:50:23,216 - root - INFO - Wrapping in: <class 'modeling_rmt.experimental.MemoryCellGenerate'> and <class 'modeling_rmt.language_modeling.RecurrentWrapper'>


In [114]:
# if args.model_cpt:
#     model_cpt = os.path.join(args.model_cpt, "model_best.pth")
#     cpt = torch.load(model_cpt, map_location='cpu')
#     model.load_state_dict(cpt['model_state_dict'], strict=False)
#     logger.info(f'Loaded RMT state dict from: {args.model_cpt}')
if args.model_cpt:
    model_cpt = os.path.join(args.model_cpt, "model_best/pytorch_model.bin")
    # model = torch.load(model_cpt, map_location='cpu')
    cpt = torch.load(model_cpt, map_location='cpu')
    # print(cpt.keys())
    model.load_state_dict(cpt, strict=False)
    logger.info(f'Loaded RMT state dict from: {args.model_cpt}')

2023-08-31 13:50:23,763 - root - INFO - Loaded RMT state dict from: /home/jovyan/rmt/runs/associative_retrieval/lr1e-03_linear_adamw_wd1e-03_k4-v16-p1-1x2048_mem4_bs512_regular_bptt-1/run_4/


In [115]:
# gen = iter(train_dataloader)
# batch = next(gen)

batch = collate_fn([train_dataset[i] for i in range(4)])
# batch_valid = collate_valid([valid_dataset[i] for i in range(4)])

In [116]:
gen_input = batch.pop('input_ids_generate')
gen_attn_mask = batch.pop('attention_mask_generate')

In [124]:
batch['input_ids'][:, -18]

tensor([101, 101, 101, 101])

In [131]:
labels = torch.stack([ids[m] for ids, m in zip(batch['input_ids'], batch['labels_mask'])])
labels

tensor([[101,  13,   4,   9,  11,  12,  12,  14,   8,  14,   0,  15,  13,   0,
           1,  10,   3, 102],
        [101,   4,  10,  14,   0,   5,  11,   0,   3,  10,   2,  14,  13,   9,
          15,   9,   7, 102],
        [101,  12,   8,   1,   8,   5,   7,   4,  13,   5,   0,   8,   8,   1,
          10,  12,   6, 102],
        [101,   6,   6,  12,  10,   5,   2,   8,  14,  10,   7,   8,  14,  10,
           9,   8,  12, 102]])

In [123]:
batch['labels_mask']

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  Tru

forward

In [117]:
out = model(**batch)
out.loss

tensor(0.1793, grad_fn=<NllLossBackward0>)

In [137]:
out.logits.argmax(dim=-1)[:, -18:-1] - labels[:, 1:]

tensor([[ -3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0],
        [  6,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0],
        [-12,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0],
        [  4,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0]])

In [139]:
labels[:, :]

tensor([[101,  13,   4,   9,  11,  12,  12,  14,   8,  14,   0,  15,  13,   0,
           1,  10,   3, 102],
        [101,   4,  10,  14,   0,   5,  11,   0,   3,  10,   2,  14,  13,   9,
          15,   9,   7, 102],
        [101,  12,   8,   1,   8,   5,   7,   4,  13,   5,   0,   8,   8,   1,
          10,  12,   6, 102],
        [101,   6,   6,  12,  10,   5,   2,   8,  14,  10,   7,   8,  14,  10,
           9,   8,  12, 102]])

In [104]:
# batch['input_ids']

In [78]:
segmented = model.segment(input_ids=batch['input_ids'])

In [79]:
segmented[0]['input_ids'].shape

torch.Size([4, 22])

In [118]:
generated = model.memory_cell.model.generate(input_ids=gen_input, max_new_tokens=100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.


In [119]:
generated

tensor([[ 10,   8,   9,   4, 100,  13,   4,   9,  11,  12,  12,  14,   8,  14,
           0,  15,  13,   0,   1,  10,   3, 102,  10,   8,   9,   4, 101,  10,
           4,   9,  11,  12,  12,  14,   8,  14,   0,  15,  13,   0,   1,  10,
           3, 102],
        [ 15,  10,  11,  13, 100,   4,  10,  14,   0,   5,  11,   0,   3,  10,
           2,  14,  13,   9,  15,   9,   7, 102,  15,  10,  11,  13, 101,  10,
          10,  14,   0,   5,  11,   0,   3,  10,   2,  14,  13,   9,  15,   9,
           7, 102],
        [ 14,  12,   0,   7, 100,  12,   8,   1,   8,   5,   7,   4,  13,   5,
           0,   8,   8,   1,  10,  12,   6, 102,  14,  12,   0,   7, 101,   0,
           8,   1,   8,   5,   7,   4,  13,   5,   0,   8,   8,   1,  10,  12,
           6, 102],
        [  3,  11,  13,  13, 100,   6,   6,  12,  10,   5,   2,   8,  14,  10,
           7,   8,  14,  10,   9,   8,  12, 102,   3,  11,  13,  13, 101,  10,
           6,  12,  10,   5,   2,   8,  14,  10,   7,   8,  14,  10,   

In [120]:
batch['labels'][:, -generated.shape[1]:] - generated

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0, -6,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0, 12,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0, -4,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0]])

gen with embeds

In [88]:
self =model
input_ids = gen_input
attention_mask = None

generate_kwargs = {'max_new_tokens': 100}

In [90]:
memory_state = None
segmented = self.segment(input_ids=input_ids, attention_mask=attention_mask)

for seg_num, segment in enumerate(segmented[:-1]):
    cell_out, memory_state = self.memory_cell(**segment, memory_state=memory_state, output_hidden_states=True)

final_segment = segmented[-1]
out = self.memory_cell.generate(**final_segment, memory_state=memory_state, attention_mask=None, **generate_kwargs)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.


In [91]:
out

tensor([[101,  14,  11,  14,  14,  11,  14,   6,  11,  14,   6,  11,  14,   6,
          11,  14,  10,  10,  10,   6,  14,  11,  11,  14,  11,  14,  14,  11,
          14,   6,  11,  14,   6,  11,  14,   6,  11,  14,  10,  10,  10,   6,
          14,  11,  11,  14,  11,  14,  14,  11,  14,   6,  11,  14,   6,  11,
          14,   6,  11,  14,  10,  10,  10,   6,  14,  11,  11,  14,  11,  14,
          14,  11,  14,   6,  11,  14,   6,  11,  14,   6,  11,  14,  10,  10,
          10,   6,  14,  11,  11,  14,  11,  14,  14,  11,  14,   6,  11,  14,
           6,  11,  14],
        [101,   1,   1,   1,   1,   1,   1,  11,   1,   1,  11,   1,   1,  11,
           1,   1,   1,   1,   1,  11,   8,   1,   1,   1,   1,   1,   1,   1,
           1,  11,   1,   1,  11,   1,   1,  11,   1,   1,   1,   1,   1,  11,
           8,   1,   1,   1,   1,   1,   1,   1,   1,  11,   1,   1,  11,   1,
           1,  11,   1,   1,   1,   1,   1,  11,   8,   1,   1,   1,   1,   1,
           1,   1,   1,  11

In [86]:
model.memory_cell.model.config.bos_token = None

In [58]:
inputs_embeds = model.memory_cell.model.get_input_embeddings()(gen_input)

In [59]:
gen_input.shape

torch.Size([4, 27])

In [60]:
inputs_embeds.shape

torch.Size([4, 27, 128])

In [87]:
generated = model.memory_cell.model.generate(inputs_embeds=inputs_embeds, max_new_tokens=50)

TypeError: generate() missing 2 required positional arguments: 'input_ids' and 'attention_mask'

In [65]:
generated

tensor([[101,  10,  14,   3,  12,  15,  13,   4,   0,   4,  10,  10,   6,   9,
           4,  13,  14, 102],
        [101,  12,   2,   1,   5,  11,   9,  12,  12,   0,  10,  12,   7,  12,
           3,  12,   4, 102],
        [101,  14,   5,  11,   3,   5,   8,   9,  13,  11,  10,   7,   5,  11,
          13,   9,   6, 102],
        [101,   4,   8,  12,   2,   0,   4,   9,  14,  12,  10,   6,   0,   3,
          13,   9,   5, 102]])

In [66]:
batch['labels'][:, -generated.shape[1]:] - generated

tensor([[  0,  -9,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0, -13,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,  -2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0]])

In [63]:
batch['labels'][0]

tensor([ 10,   8,  11,   7, 100,   1,  14,   3,  12,  15,  13,   4,   0,   4,
         10,  10,   6,   9,   4,  13,  14, 102,  10,   8,  11,   7, 101,   1,
         14,   3,  12,  15,  13,   4,   0,   4,  10,  10,   6,   9,   4,  13,
         14, 102])

In [212]:
generated = model.memory_cell.model.generate(inputs_embeds=inputs_embeds, max_new_tokens=17, pad_token_id=104, eos_token_id=102)

In [215]:
gen_input[0]

tensor([ 10,   2,   7,  11, 100,   2,   3,   4,  11,   3,   6,   5,   3,  13,
         12,   0,   7,  13,   6,   3,  14, 102,  10,   2,   7,  11, 101])

In [217]:
batch['labels'][0]

tensor([ 10,   2,   7,  11, 100,   2,   3,   4,  11,   3,   6,   5,   3,  13,
         12,   0,   7,  13,   6,   3,  14, 102,  10,   2,   7,  11, 101,   2,
          3,   4,  11,   3,   6,   5,   3,  13,  12,   0,   7,  13,   6,   3,
         14, 102])

In [213]:
generated

tensor([[101,   2,   3,   4,  11,   3,   6,   5,   3,  13,  12,   0,   7,  13,
           6,   3,  14, 102],
        [101,  12,   8,   2,   3,   9,   2,   5,  10,   0,  15,   4,   1,  15,
           7,  15,  12, 102],
        [101,  10,  15,  12,  13,  13,  13,   9,   6,   0,   9,  14,  11,   0,
          12,  12,   6, 102],
        [101,   7,   8,   7,   0,   0,   5,  11,   1,  14,  11,  13,  11,  14,
           8,   8,   6, 102]])

In [209]:
model.memory_cell.model.generate(max_new_tokens=17)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.


tensor([[101,   3,   5,   3,   5,   5,   3,   5,   5,   3,   5,   3,   5,   3,
           5,   5,   3,   5]])

In [140]:
y = torch.stack([l[m] for l, m in zip(batch['labels'], batch['labels_mask'])])
p = generated

In [141]:
y

tensor([[101,  12,   7,   6,   1,  11,   8,  15,   0,   7,   4,  12,  13,   7,
           8,   9,   8, 102],
        [101,   1,   1,   0,  12,  11,   6,   9,   7,  14,  11,   3,  13,   4,
           1,   9,  14, 102],
        [101,   4,  13,   0,   1,   7,  14,   1,  11,   9,   2,   5,   3,   1,
           7,  10,   1, 102],
        [101,   8,   5,   9,   8,  12,   7,  12,  13,   8,  11,   1,  11,   5,
           3,  12,   0, 102]])

In [142]:
p[:, -y.shape[1]:]  - y

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, -3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [116]:
generated[:, -y.shape[1]:]

tensor([[ 12,   8,   8,   9,   8,   5,  12,   7,   7,  14,  11,   1,  12,   0,
          12,  14, 102],
        [ 12,   6,  13,  13,   2,   3,   8,   6,   1,   3,   0,   7,   2,  14,
           5,  15, 102],
        [  9,   9,   2,   2,   9,  14,   3,   1,   0,   4,   0,   7,   8,  14,
           6,   5, 102],
        [ 12,   5,   6,  14,  12,   6,  15,   4,   2,  10,  13,  15,  14,   7,
           1,   8, 102]])

In [122]:
y

tensor([[  5,   8,   8,   9,   8,   5,  12,   7,   7,  14,  11,   1,  12,   0,
          12,  14, 102],
        [  8,   6,  13,  13,   2,   3,   8,   6,   1,   3,   0,   7,   2,  14,
           5,  15, 102],
        [  7,   9,   2,   2,   9,  14,   3,   1,   0,   4,   0,   7,   8,  14,
           6,   5, 102],
        [  7,   5,   6,  14,  12,   6,  15,   4,   2,  10,  13,  15,  14,   7,
           1,   8, 102]])

In [119]:
y

tensor([[  5,   8,   8,   9,   8,   5,  12,   7,   7,  14,  11,   1,  12,   0,
          12,  14, 102],
        [  8,   6,  13,  13,   2,   3,   8,   6,   1,   3,   0,   7,   2,  14,
           5,  15, 102],
        [  7,   9,   2,   2,   9,  14,   3,   1,   0,   4,   0,   7,   8,  14,
           6,   5, 102],
        [  7,   5,   6,  14,  12,   6,  15,   4,   2,  10,  13,  15,  14,   7,
           1,   8, 102]])

In [111]:
y

tensor([[  5,   8,   8,   9,   8,   5,  12,   7,   7,  14,  11,   1,  12,   0,
          12,  14, 102],
        [  8,   6,  13,  13,   2,   3,   8,   6,   1,   3,   0,   7,   2,  14,
           5,  15, 102],
        [  7,   9,   2,   2,   9,  14,   3,   1,   0,   4,   0,   7,   8,  14,
           6,   5, 102],
        [  7,   5,   6,  14,  12,   6,  15,   4,   2,  10,  13,  15,  14,   7,
           1,   8, 102]])

In [107]:
em  = np.mean([y_ == p_[-len(y_):] for p_, y_ in zip (p, y)])
em


0.9411764705882353

In [103]:
gen_input.shape[0]

4

In [98]:
generated

tensor([[  7,  13,   0,  10, 100,   5,   8,   8,   9,   8,   5,  12,   7,   7,
          14,  11,   1,  12,   0,  12,  14, 102,   7,  13,   0,  10, 101,  12,
           8,   8,   9,   8,   5,  12,   7,   7,  14,  11,   1,  12,   0,  12,
          14, 102],
        [ 14,   6,   5,   5, 100,   8,   6,  13,  13,   2,   3,   8,   6,   1,
           3,   0,   7,   2,  14,   5,  15, 102,  14,   6,   5,   5, 101,  12,
           6,  13,  13,   2,   3,   8,   6,   1,   3,   0,   7,   2,  14,   5,
          15, 102],
        [  9,  12,  10,   8, 100,   7,   9,   2,   2,   9,  14,   3,   1,   0,
           4,   0,   7,   8,  14,   6,   5, 102,   9,  12,  10,   8, 101,   9,
           9,   2,   2,   9,  14,   3,   1,   0,   4,   0,   7,   8,  14,   6,
           5, 102],
        [  1,   4,   3,  14, 100,   7,   5,   6,  14,  12,   6,  15,   4,   2,
          10,  13,  15,  14,   7,   1,   8, 102,   1,   4,   3,  14, 101,  12,
           5,   6,  14,  12,   6,  15,   4,   2,  10,  13,  15,  14,   

tensor(0., grad_fn=<NllLossBackward0>)

In [58]:
generated

tensor([[  2,  15,  12,  10, 100,   4,   8,   6,   2,   3,   3,   2,   0,  15,
          11,   3,  15,   6,  13,   0,  13, 102,   2,  15,  12,  10, 101,   9,
          32, 100,  80,  80,  80,  80,  80,  80,  80,  80,  80,  73,  59,  70,
          64,  70,  64,  70,  64,  70,  64,  68, 117,   9,  32, 120, 110,  41,
         111, 126,  58,  71,  43,  71,  43,   8, 112, 103, 120, 110,  94, 126,
          95,  97,   9,  32, 100,  43,   8],
        [  4,   0,   5,   1, 100,  13,  10,  13,   2,   3,   3,  14,   7,  11,
           8,   6,  15,  10,  12,   9,  11, 102,   4,   0,   5,   1, 101,   9,
          71,  71,  71,  71,  71,  43, 100,  43, 100,  43, 100,  43, 100,  92,
          80, 107,   1,  58,  71,  43, 100,  92,  80, 107,   1,  58,  71,  43,
         100,  92,  80, 107,   1,  58,  71,  43, 100,  92,  80, 107,   1,  58,
          71,  43, 100,  92,  80, 107,   1],
        [  6,  11,  11,   1, 100,   5,  14,  14,   1,   5,   1,  13,  13,   5,
           1,   2,  11,   8,   9,  10,   

In [ ]:
tokenizer.batch_decode(generated)

["Receiving Party may share some Confidential Information with some of Receiving Party's employees. Attachment 8\nNon-Disclosure Agreement for Request for Proposal – California Courts Protective Order Registry Development\nTHIS CONFIDENTIALITY AGREEMENT (“Agreement”), effective as of _________________[date of company’s signed proposal] (“Effective Date”), is entered into by and between the Administrative Office of the Courts (the “AOC”) and [company name], a [ ] corporation, with its principal address at [complete address], (hereinafter the “Company”).\nWHEREAS, the State of California, acting through the Judicial Council of California, Administrative Office of the Courts is planning to disclose certain confidential information to potential vendors who wish to bid on the Request for Proposal for Judicial Council of California Administrative Office of the Courts (AOC) for California Courts Protective Order Registry Development, including all Statements of Work, dated on or about April, 

In [ ]:
tokenizer.batch_decode(labels)

['Entailment<|endoftext|><|endoftext|>', 'NotMentioned<|endoftext|>']

In [ ]:
self = model
input_ids = batch['input_ids']
attention_mask = batch['attention_mask']

In [ ]:
memory_state = None
segmented = self.segment(input_ids=input_ids, attention_mask=attention_mask)

cell_outputs = []
for seg_num, segment in enumerate(segmented[:-1]):
    cell_out, memory_state = self.memory_cell(**segment, memory_state=memory_state, output_hidden_states=True)
    cell_outputs.append(cell_out)
    # self.manage_gradients(memory_state, seg_num)

final_segment = segmented[-1]
out = self.memory_cell.generate(**final_segment, memory_state=memory_state)

ModuleAttributeError: 'MemoryCell' object has no attribute 'generate'

In [ ]:
kwargs = dict(**final_segment)
input_ids = kwargs.pop('input_ids')

In [ ]:
seg_kwargs = self.memory_cell.process_input(input_ids, memory_state, **kwargs)
out = self.memory_cell.model.generate(**seg_kwargs)
# out, new_memory_state = self.memory_cell.process_output(out, **kwargs)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ValueError: If inputs_embeds is passed as model-specific keyword input then model has to be an encoder-decoder and not a GPT2LMHeadModel.

In [ ]:
seg_kwargs['inputs_embeds'].shape

torch.Size([2, 16, 768])

In [ ]:
model.memory_cell.model.generate(seg_kwargs['inputs_embeds'], max_new_tokens=20)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RuntimeError: shape '[-1, 768]' is invalid for input of size 32

In [ ]:
len(cell_outputs), len(segmented)

(4, 5)

In [ ]:
len

In [ ]:
batch['input_ids'].shape

torch.Size([2, 508])

In [ ]:
tokenizer.batch_decode(batch['input_ids'])

["Receiving Party may share some Confidential Information with some of Receiving Party's employees. Attachment 8\nNon-Disclosure Agreement for Request for Proposal – California Courts Protective Order Registry Development\nTHIS CONFIDENTIALITY AGREEMENT (“Agreement”), effective as of _________________[date of company’s signed proposal] (“Effective Date”), is entered into by and between the Administrative Office of the Courts (the “AOC”) and [company name], a [ ] corporation, with its principal address at [complete address], (hereinafter the “Company”).\nWHEREAS, the State of California, acting through the Judicial Council of California, Administrative Office of the Courts is planning to disclose certain confidential information to potential vendors who wish to bid on the Request for Proposal for Judicial Council of California Administrative Office of the Courts (AOC) for California Courts Protective Order Registry Development, including all Statements of Work, dated on or about April, 

In [ ]:
batch['attention_mask']

tensor([[ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True, False]])

In [ ]:
tokenizer.batch_decode(batch['labels'])

['Entailment<|endoftext|><|endoftext|>', 'NotMentioned<|endoftext|>']

In [ ]:
cpt = torch.load("../../runs/cnli_original/gpt2/lr5e-05_linear_adamw_wd1e-03_124-128-1x128_mem2_bs32_iters6000_regular_bptt-1_refactor/run_1/model_best.pth", map_location='cpu')
model.load_state_dict(cpt['model_state_dict'])

<All keys matched successfully>

In [ ]:
# for i in range(10):
#     out = model(**batch)
#     out.loss.backward()
#     optimizer.step()

In [ ]:
batch['input_ids'][0][batch['labels_mask'][0]]

tensor([50257,  3673,    44,  1463,   276])

In [ ]:
batch = next(gen)

In [ ]:
out = model(**batch)

In [ ]:
out.loss

tensor(1.3551, grad_fn=<NllLossBackward>)

In [ ]:
pred = out.logits.argmax(dim=-1)


In [ ]:
tokenizer.batch_decode([p[m] for p, m in zip(pred, batch['labels_mask'])], add_special_tokens=False)

['Notailment<|endoftext|>', 'Notailment<|endoftext|>']

In [ ]:
tokenizer.batch_decode([p[m] for p, m in zip(batch['labels'], batch['labels_mask'])], add_special_tokens=False)

['[GEN]Entailment', '[GEN]Entailment']

In [ ]:
tokenizer.batch_decode([p[m] for p, m in zip(pred, batch['labels_mask'])], add_special_tokens=False)

['NotMentioned<|endoftext|>', 'Notailment<|endoftext|>']

In [ ]:
tokenizer.batch_decode([p[m] for p, m in zip(pred, batch['labels_mask'])], add_special_tokens=False)

['NotMentioned<|endoftext|>', 'NotMentionedNot']

In [ ]:
out.loss

tensor(0.2207, grad_fn=<NllLossBackward>)

In [ ]:
gen_inputs = [b[~m] for b, m in zip(batch['input_ids'], batch['labels_mask'])]

In [ ]:
tokenizer.decode(gen_inputs[0])

'Receiving Party shall not disclose the fact that Agreement was agreed or negotiated. Johns Hopkins University\nNON-DISCLOSURE AGREEMENT For Bilateral Disclosure\nThis Agreement is effective this of in the year ______ is by and between JHU and COMPANY, each defined below.\nJHU: The Johns Hopkins University\nAddress: 100 N. Charles St., 5th Floor\nBaltimore, Maryland 21201\nJHU Contact:\nCOMPANY:\nAddress:\nCOMPANY Contact:\nWHEREAS, each party has certain technical information described below which shall hereinafter be referred to as "CONFIDENTIAL INFORMATION";\nCONFIDENTIAL INFORMATION:\nWHEREAS, JHU and COMPANY are each interested in examining the CONFIDENTIAL INFORMATION of the other solely for the PURPOSE, defined below;\nPURPOSE: To explore licensing, collaborative or sponsored research agreement opportunities related to the CONFIDENTIAL INFORMATION.\nNOW, THEREFORE, in consideration of the premises and mutual covenants contained herein, the parties hereto agree as follows:\n”PROV

In [ ]:
gt = tokenizer.batch_decode([l[m] for l, m in zip(batch['labels'], batch['labels_mask'])], skip_special_tokens=True)

In [ ]:
gt

['Entailment', 'Entailment']

In [ ]:
accuracy_score(pred, pred)

1.0

In [ ]:
tokenizer.batch_decode([l[m] for l, m in zip(batch['labels'], batch['labels_mask'])], skip_special_tokens=True)

['NotMentioned', 'Contradiction']

In [ ]:
out.loss

tensor(62.5257, grad_fn=<NllLossBackward>)

In [ ]:
for i in range(15):
    out = model(**batch)
    out.loss.backward()
    optimizer.step()

In [ ]:
out.loss

tensor(7.8596, grad_fn=<NllLossBackward>)

In [ ]:
pred = out.logits.argmax(dim=-1)

In [ ]:
pred_text = tokenizer.batch_decode(pred)

In [ ]:
generated = model.memory_cell.model.generate(batch['labels'][:, -5:])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
batch['input_ids'][0][batch['labels_mask'][0]]

tensor([50257,  3673,    44,  1463,   276, 50256])

In [ ]:
batch['labels'][0][batch['labels_mask'][0]]

tensor([50257,  3673,    44,  1463,   276, 50256])

In [ ]:
gen_inputs = [torch.cat((b[~m], torch.tensor([gen_token]))) for b, m in zip(batch['input_ids'], batch['labels_mask'])]

In [ ]:
gen = model.memory_cell.model.generate(gen_inputs)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
output = generated

In [ ]:
 def keep_for_metrics_fn(batch, output):
    data = {}
    if isinstance(output, torch.Tensor):
        data['labels'] = batch['labels']
        
    for key in batch.keys():
        if 'loss' in key: 
            data[key] = batch[key]
    return data

In [ ]:
tokenizer.batch_decode(generated, add_special_tokens=False)

['[GEN]NotMentioned[GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN]',
 ' in[GEN]Contradiction[GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN][GEN]']

In [ ]:
#  def keep_for_metrics_fn(batch, output):
data = {}
# if 'generation_outputs' in output:
if isinstance(output, dict):
    data['predictions'] = torch.argmax(output['logits'].detach(), dim=-1)
    if 'labels_mask' in batch:
        data['predictions'] = [data['predictions'][i, mask] for i, mask in enumerate(batch['labels_mask'])]
else:
    data['generation_outputs'] = output
    
data['labels'] = batch['labels']
for key in batch.keys():
    if 'loss' in key: 
        data[key] = batch[key]

In [ ]:
# out = CausalLMOutputWithCrossAttentions()
# full_logits = torch.cat([o.logits for o in cell_outputs], dim=1)
# full_hidden_states = tuple([torch.cat(layer_hs, dim=1) for layer_hs in zip(*[o.hidden_states for o in cell_outputs])])

full_logits = out.logits
labels = kwargs.get('labels')
if labels is not None:
    shift_labels = labels[..., 1:].contiguous()
    shift_logits = full_logits[..., :-1, :].contiguous()
    flat_labels = shift_labels.view(-1)
    flat_logits = shift_logits.view(-1, shift_logits.size(-1))
    
    loss_fct = CrossEntropyLoss()
    labels_mask = kwargs.get('labels_mask')
    if labels_mask is not None:
        shift_mask = labels_mask[..., :-1].contiguous()

        flat_labels = flat_labels[shift_mask.view(-1)]
        flat_logits = flat_logits[shift_mask.view(-1)]
        
    out['loss'] = loss_fct(flat_logits, flat_labels)

out['logits'] = full_logits
segment_keys = ['loss', 'logits']
if kwargs.get('output_attentions'):
    segment_keys.append('attentions')
if kwargs.get('output_hidden_states'):
    segment_keys.append('hidden_states')
    out['hidden_states'] = full_hidden_states

In [ ]:

### booydar
batch_metrics_fn = lambda _, y: {key: y[key] for key in y.keys() if (('loss' in key) or ('!log' in key))}
trainer = Trainer(args, model, optimizer, train_dataloader, valid_dataloader, train_sampler,
                    keep_for_metrics_fn=keep_for_metrics_fn, metrics_fn=metrics_fn,
                    ###booydar
                    batch_metrics_fn=batch_metrics_fn,
                    generate_kwargs={})

if not args.validate_only:
    # train loop
    trainer.train()
    # make sure all workers are done
    hvd.barrier()
    # run validation after training
    if args.save_best:
        best_model_path = str(Path(args.model_path) / 'model_best.pth')
        if hvd.rank() == 0:
            logger.info(f'Loading best saved model from {best_model_path}')
        trainer.load(best_model_path)
    if valid_dataloader is not None:
        if hvd.rank() == 0:
            logger.info('Runnning validation on valid data:')
        trainer.validate(valid_dataloader, write_tb=False)
else:
    # run validation, do not write to tensorboard
    if hvd.rank() == 0:
        logger.info('Running validation on train set:')
    trainer.validate(train_dataloader, split='train', write_tb=True)
    if valid_dataloader is not None:
        if hvd.rank() == 0:
            logger.info('Running validation on valid data:')
        trainer.validate(valid_dataloader, write_tb=True)
    # if test_dataloader is not None:
    #     if hvd.rank() == 0:
    #         logger.info('Runnning validation on test data:')
    #     trainer.validate(test_dataloader, write_tb=True)


NameError: name 'train_sampler' is not defined

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

### Finetune

In [ ]:
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import sys
sys.path.append('..')

### load dataset 

In [ ]:
class Holder:
    def __init__(self):
        pass

In [ ]:
input_seq_len = 512
target_seq_len = 512

num_mem_tokens = 2
input_size = 128

batch_size = 2

args = Holder
args.target_seq_len = target_seq_len
args.input_seq_len = input_seq_len
args.num_mem_tokens = num_mem_tokens
args.input_size = input_size
args.input_prefix = ''
args.block_size = None
args.task_name = 'wikitext-2-v1'

device = 'cpu'

In [ ]:
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [ ]:
from itertools import chain

raw_datasets = datasets.load_dataset('wikitext', args.task_name)
column_names = raw_datasets["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]

def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=column_names,
    desc="Running tokenizer on dataset",
)

block_size = args.input_size 
if args.num_mem_tokens is not None:
    block_size -= 2 * args.num_mem_tokens
history_size = args.input_seq_len - block_size

def group_texts(examples, block_size, history_size=None):
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])

    if history_size is None:
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
    else:
        result = {
            k: [t[max({0, i - history_size}) : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
    result["labels"] = result["input_ids"].copy()
    return result

Found cached dataset wikitext (/home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-389b922bfc5fe729.arrow
Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-6067a66e735cfbb1.arrow
Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-941845a5470f2db7.arrow


In [ ]:
id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

In [ ]:
block_size, history_size

(124, 388)

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# def collate_fn(batch):
#     input_ids = [torch.tensor(b['input_ids'][::-1]) for b in batch]
#     labels = [torch.tensor(b['labels'][::-1]) for b in batch]
#     attention_mask = [torch.tensor(b['attention_mask'][::-1]) for b in batch]
#     input_ids = pad_sequence(input_ids, padding_value=id_pad_value).T.flip(1)
#     labels = pad_sequence(labels, padding_value=-100).T.flip(1)
#     attention_mask = pad_sequence(attention_mask, padding_value=0).T.flip(1)

#     collated = {'input_ids': input_ids,
#                 'labels': labels, 
#                 'attention_mask': attention_mask}

#     if input_ids.shape[1] != block_size:
#         labels_mask = torch.ones_like(input_ids, dtype=bool)
#         labels_mask[:, :-block_size] = False
#         collated['labels_mask'] = labels_mask

#     return collated


def collate_fn(batch):
    input_ids = [torch.tensor(b['input_ids']) for b in batch]
    labels = [torch.tensor(b['labels']) for b in batch]
    labels_mask = [torch.ones_like(l, dtype=bool) for l in labels]
    attention_mask = [torch.tensor(b['attention_mask']) for b in batch]

    input_ids = pad_sequence(input_ids, padding_value=id_pad_value).T
    labels = pad_sequence(labels, padding_value=-100).T
    labels_mask = pad_sequence(labels_mask, padding_value=False).T
    attention_mask = pad_sequence(attention_mask, padding_value=0).T

    collated = {'input_ids': input_ids,
                'labels': labels, 
                'labels_mask': labels_mask,
                'attention_mask': attention_mask}

    # if args.vary_n_segments:
    #     n_segments = np.random.randint(1, args.max_n_segments + 1)
    #     n_tokens = n_segments * block_size
    #     for k in collated:
    #         collated[k] = collated[k][:, -n_tokens:]

    return collated


train_dataset = tokenized_datasets["train"].map(lambda x: group_texts(x, block_size, history_size), 
                                        batched=True, desc=f"Grouping train in chunks of {block_size} and history {history_size}")
valid_dataset = tokenized_datasets["validation"].map(lambda x: group_texts(x, block_size), 
                                        batched=True, desc=f"Grouping valid in chunks of {block_size}")


# shuffle train data each epoch (one loop over train_dataset)
# train_sampler = DistributedStrain_dataset[i] for i in range(4)ampler(train_dataset, rank=hvd.rank(), num_replicas=hvd.size(), shuffle=True,
#                                     drop_last=False, seed=args.seed)
# per_worker_batch_size = args.batch_size * args.gradient_accumulation_steps
# global_batch_size = per_worker_batch_size * hvd.size()

# train_sampler = RandomSampler(train_dataset)
kwargs = {'pin_memory': True}#, 'num_workers': args.data_n_workers}
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, **kwargs)

Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-1504f9373e317eca.arrow
Loading cached processed dataset at /home/bulatov/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-c6da793e710ea6d8.arrow


In [ ]:
b = [train_dataset[i] for i in range(4)]

In [ ]:
for k in b[0]:
    b[0][k] = b[0][k][:124]

In [ ]:
batch = collate_fn(b)

In [ ]:
gen = iter(train_dataloader)
batch = next(gen)
batch = next(gen)
batch = next(gen)
batch['input_ids'].shape

torch.Size([2, 512])

In [ ]:
# raw_datasets['train'][1]

### Model

In [ ]:
import math
import torch
from torch.nn import CrossEntropyLoss

In [ ]:
from transformers.modeling_outputs import CausalLMOutputWithCrossAttentions

class MemoryCell(torch.nn.Module):
    def __init__(self, base_model, num_mem_tokens):
        super().__init__()
        self.model = base_model
        self.create_memory(num_mem_tokens)

    def create_memory(self, num_mem_tokens):
        self.num_mem_tokens = num_mem_tokens
        embeddings = self.model.get_input_embeddings()
        memory_weights = torch.randn((num_mem_tokens, self.model.config.n_embd)) * embeddings.weight.data.std()
        self.register_parameter('memory', torch.nn.Parameter(memory_weights, requires_grad=True))

        self.read_memory_position = range(num_mem_tokens)
        self.write_memory_position = range(-num_mem_tokens, 0)

    def set_memory(self, input_shape):
        memory = self.memory.repeat(input_shape[0], 1, 1)
        return memory

    def forward(self, input_ids, memory_state=None, **kwargs):
        if memory_state is None:
            memory_state = self.set_memory(input_ids.shape)

        seg_kwargs = self.process_input(input_ids, memory_state, **kwargs)
        out = self.model(**seg_kwargs)
        out, new_memory_state = self.process_output(out, **kwargs)

        return out, new_memory_state
    
    def process_input(self, input_ids, memory_state, **kwargs):
        seg_kwargs = dict(**kwargs)

        inputs_embeds = kwargs.get('inputs_embeds')
        if inputs_embeds is None:
            inputs_embeds = self.model.get_input_embeddings()(input_ids)
        inputs_embeds = torch.cat([memory_state, inputs_embeds, memory_state], dim=1)

        seg_kwargs['input_ids'] = None
        seg_kwargs['inputs_embeds'] = inputs_embeds
        if kwargs.get('attention_mask') is not None:
            seg_kwargs['attention_mask'] = self.pad_attention_mask(kwargs['attention_mask'], inputs_embeds.shape)
        seg_kwargs['output_hidden_states'] = True
        return seg_kwargs
    
    def pad_attention_mask(self, attention_mask, shape):
        if self.num_mem_tokens in {0, None}:
            return attention_mask
        else:
            mask = torch.ones(*shape[:2], dtype=torch.int64).to(attention_mask.device)
            mask[:, self.num_mem_tokens:-self.num_mem_tokens] = attention_mask
            return mask
    
    def process_output(self, model_outputs, **kwargs):
        if self.num_mem_tokens not in {0, None}:
            out = CausalLMOutputWithCrossAttentions()
            memory_state = model_outputs.hidden_states[-1][:, -self.num_mem_tokens:]
            out['logits'] = model_outputs.logits[:, self.num_mem_tokens:-self.num_mem_tokens]
            
            if kwargs.get('output_hidden_states'):
                out['hidden_states'] = [lh[:, self.num_mem_tokens:-self.num_mem_tokens] for lh in model_outputs.hidden_states]
            if kwargs.get('output_attentions'):
                out['attentions'] = model_outputs['attentions']
        else:
            memory_state = None
            out = model_outputs
            
        return out, memory_state 


In [ ]:
class RecurrentWrapper(torch.nn.Module):
    def __init__(self, memory_cell, **rmt_kwargs):
        super().__init__()
        self.memory_cell = memory_cell
        self.rmt_config = rmt_kwargs

    def forward(self, input_ids, labels=None, labels_mask=None, inputs_embeds=None, attention_mask=None, output_attentions=None, output_hidden_states=None):
        memory_state = None
        segmented = self.segment(input_ids=input_ids, inputs_embeds=inputs_embeds, attention_mask=attention_mask)

        cell_outputs = []
        for seg_num, segment in enumerate(segmented):
            cell_out, memory_state = self.memory_cell(**segment, memory_state=memory_state, output_hidden_states=True)
            cell_outputs.append(cell_out)
            self.manage_gradients(memory_state, seg_num)

        out = self.process_outputs(cell_outputs, labels=labels, 
                                   labels_mask=labels_mask,
                                   output_attentions=output_attentions, 
                                   output_hidden_states=output_hidden_states)
        return out

    def segment(self, **kwargs):
        segments = []
        for k, tensor in kwargs.items():
            if tensor is not None:
                k_segments = self.split_tensor(tensor)
                for s, k_seg in enumerate(k_segments):
                    if s < len(segments):
                        segments[s][k] = k_seg
                    else:
                        segments.append({k: k_seg})

        return segments
    
    def split_tensor(self, tensor):
        align = self.rmt_config.get('segment_alignment')
        segment_size = self.rmt_config.get('segment_size')
        if align in {'left', None}:
            split_inds = list(range(0, tensor.shape[1], segment_size)) + [tensor.shape[1]]
            segments = [tensor[:, start:end] for (start, end) in zip(split_inds, split_inds[1:])]
        elif align in {'right', None}:
            split_inds = (list(range(tensor.shape[1], 0, -segment_size)) + [0])[::-1]
            segments = [tensor[:, start:end] for (start, end) in zip(split_inds, split_inds[1:])]
        elif align == 'center':
            n_seg = math.ceil(tensor.shape[1] / segment_size)
            segments = torch.chunk(tensor, n_seg, dim=1)
        else:
            raise NotImplementedError
        return segments

    def process_outputs(self, cell_outputs, **kwargs):
        out = CausalLMOutputWithCrossAttentions()
        full_logits = torch.cat([o.logits for o in cell_outputs], dim=1)
        full_hidden_states = tuple([torch.cat(layer_hs, dim=1) for layer_hs in zip(*[o.hidden_states for o in cell_outputs])])

        labels = kwargs.get('labels')
        if labels is not None:
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = full_logits[..., :-1, :].contiguous()
            flat_labels = shift_labels.view(-1)
            flat_logits = shift_logits.view(-1, shift_logits.size(-1))
            
            loss_fct = CrossEntropyLoss()
            labels_mask = kwargs.get('labels_mask')
            if labels_mask is not None:
                shift_mask = labels_mask[..., :-1].contiguous()

                flat_labels = flat_labels[shift_mask.view(-1)]
                flat_logits = flat_logits[shift_mask.view(-1)]
                
            out['loss'] = loss_fct(flat_logits, flat_labels)

        out['logits'] = full_logits
        segment_keys = ['loss', 'logits']
        if kwargs.get('output_attentions'):
            segment_keys.append('attentions')
        if kwargs.get('output_hidden_states'):
            segment_keys.append('hidden_states')
            out['hidden_states'] = full_hidden_states

        # for seg_num, o in enumerate(cell_outputs):
        #     for key, value in o.items():
        #         if any([sk in key for sk in segment_keys]):
        #             out[f'{key}_{seg_num}'] = value

        return out 
        
    def manage_gradients(self, memory_state, seg_num):
        k2, max_n_segments = self.rmt_config.get('k2'), self.rmt_config.get('max_n_segments')
        if seg_num == 0 \
            or k2 in {-1, None} \
            or seg_num + k2 > max_n_segments:
                return True
        
        memory_state = memory_state.detach()
        return False

In [ ]:

num_mem_tokens = 10
# device = torch.device(3)
device = 'cpu'

rmt_config = {'num_mem_tokens': 10, 
               #  'max_n_segments': 1,
               #  'segment_alignment': 'right',
               #  'tokenizer': tokenizer,
               #  'memory_layers': 'all', 
               #  'share_memory_layers': True,
               #  'reconstruction_loss_coef': 0.1,
               #  'k1': -1, 'k2': 3,
               #  'segment_ordering': 'regular',
               #  'input_size': 1024, 
               #  'bptt_depth': -1, 
               #  'sum_loss': False,
             }

base_model = AutoModelForCausalLM.from_pretrained(model_name)
cell = MemoryCell(base_model, num_mem_tokens=2)

/cephfs/home/bulatov/bulatov/hvdenv/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [ ]:
rmt = RecurrentWrapper(cell, max_n_segments=5, segment_size=124, segment_alignment='center')

In [ ]:
gen = iter(train_dataloader)
batch = next(gen)
batch = next(gen)
batch = next(gen)
batch.pop('labels_mask')
# batch.pop('labels')
1

1

In [ ]:
rmt_out = rmt(**batch)

In [ ]:
rmt_out.keys()

odict_keys(['loss', 'logits'])

# Contract-NLI

In [ ]:
from datasets import Dataset

class CNLIDataset(Dataset):
    def __init__(self, json_path):
        with open(json_path, 'r') as f:
            self.samples = json.load(f)

    def __getitem__(self, item):
        return self.samples[item]
    
    def __len__(self):
        return len(self.samples)

In [ ]:
dataset = CNLIDataset('/cephfs/home/bulatov/bulatov/datasets/contract_nli/contract-nli/test_processed.json')

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0032s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0160s.) Setting batch_size=4.
[Parallel(n_jobs=12)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  50 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 123 out of 123 | elapsed:    0.1s finished


In [ ]:
args.input_seq_len

512

In [ ]:
from torch.nn.utils.rnn import pad_sequence

tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_tokens('[GEN]', special_tokens=True)
gen_token = tokenizer.encode('[GEN]')[0]
id_pad_value = tokenizer.eos_token_id

def collate_fn(batch):
    inputs = [b['context'][:args.input_seq_len * 10] for b in batch]
    labels = [b['answer'][:args.input_seq_len * 10] for b in batch]

    collated = {}
    inputs = tokenizer.batch_encode_plus(list(inputs), padding=False)
    labels = tokenizer.batch_encode_plus(list(labels), padding=False)

    full_inputs, labels_mask = [], []
    for inp, lab in zip(inputs['input_ids'], labels['input_ids']):
        inp = inp[:args.input_seq_len - len(lab) - 1]
        full_inputs.append(torch.tensor(inp + [gen_token] + lab))
        labels_mask.append(torch.tensor([False] * len(inp) + [True] * (len(lab) + 1)))

    full_inputs = pad_sequence(full_inputs, padding_value=id_pad_value).T
    labels_mask = pad_sequence(labels_mask, padding_value=False).T

    collated['input_ids'] = collated['labels'] = full_inputs
    collated['labels_mask'] = labels_mask
    collated['attention_mask'] = collated['input_ids'] != id_pad_value
    return collated

In [ ]:
batch = collate_fn([dataset[i] for i in range(4)])
# batch = [dataset[i] for i in range(4)]

In [ ]:
segmented = rmt.segment(**batch)

In [ ]:
[s['input_ids'].shape for s in segmented]

[torch.Size([4, 103]),
 torch.Size([4, 103]),
 torch.Size([4, 103]),
 torch.Size([4, 103]),
 torch.Size([4, 100])]

In [ ]:
labels = batch.pop('labels')
rmt_out = rmt(**batch)

In [ ]:
rmt_out.logits.shape

torch.Size([4, 512, 50258])

In [ ]:
rmt_out.logits[batch['labels_mask']].shape

torch.Size([20, 50258])

In [ ]:
labels[batch['labels_mask']].shape

torch.Size([20])

In [ ]:
tokenizer.decode([ 3673,    44,  1463,   276, 50256])

'NotMentioned<|endoftext|>'

In [ ]:
tokenizer.decode([14539,   603,   434])

'Entailment'

In [ ]:
# rmt_out = rmt(**batch)

self = rmt
input_ids = batch['input_ids']

memory_state = None
segmented = self.segment(input_ids=input_ids, inputs_embeds=None, attention_mask=None)

cell_outputs = []
for seg_num, segment in enumerate(segmented):
    cell_out, memory_state = self.memory_cell(**segment, memory_state=memory_state, output_hidden_states=True)#**batch)
    cell_outputs.append(cell_out)
    self.manage_gradients(memory_state, seg_num)

# out = self.process_outputs(cell_outputs)

In [ ]:
input_ids.shape

torch.Size([2, 248])

In [ ]:
torch.chunk(torch.ones(10), 3)

(tensor([1., 1., 1., 1.]), tensor([1., 1., 1., 1.]), tensor([1., 1.]))

### seq2seq

In [ ]:
from huggingface_hub import hf_hub_download
# scrolls_metric_path = hf_hub_download(repo_id="datasets/tau/scrolls", filename="metrics/scrolls.py")tokenizer.add_tokens('[GEN]', special_tokens=True)
gen_token = tokenizer.encode('[GEN]')[0]

rmt.memory_cell.model.resize_token_embeddings(len(tokenizer))

# gen_token = tokenizer.eos_token_id

Embedding(50258, 768)

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [ ]:
global_attention_first_token = False  # should be True for LED
encode_plus_kwargs = {'truncation': True, 'padding': 'longest', 'pad_to_multiple_of': 1}
# generate_kwargs = {'max_length': args.targettokenizer.add_tokens('[GEN]', special_tokens=True)
# gen_token = tokenizer.encode('[GEN]')[0]

# rmt.memory_cell.model.resize_token_embeddings(len(tokenizer))_seq_len, 'min_length': args.target_seq_len}
generate_kwargs = {}

from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    inputs = [b['input'][:args.input_seq_len * 10] for b in batch]
    labels = [b['output'][:args.input_seq_len * 10] for b in batch]

    collated = {}
    inputs = tokenizer.batch_encode_plus(list(inputs), padding=False)
    labels = tokenizer.batch_encode_plus(list(labels), padding=False)

    full_inputs = [torch.tensor(i[:input_size - len(l) - 1] + [gen_token] + l) for i, l in zip(inputs['input_ids'], labels['input_ids'])]
    full_inputs = pad_sequence(full_inputs, padding_value=tokenizer.pad_token_id).T
    
    labels_mask = torch.zeros_like(full_inputs).bool()
    for i, l in enumerate(labels['input_ids']):
        labels_mask[i, -len(l) -1:] = True
    collated['input_ids'] = collated['labels'] = full_inputs
    collated['labels_mask'] = labels_mask
    collated['attention_mask'] = collated['input_ids'] != tokenizer.pad_token_id

    return collated
    

In [ ]:
seq2seq_task_name = 'quality'
dataset = datasets.load_dataset('tau/scrolls', seq2seq_task_name)
train_dataset = dataset['train']
# shuffle train data each epoch (one loop over train_dataset)
# train_sampler = DistributedSampler(train_dataset, shuffle=True, drop_last=False, seed=args.seed)
kwargs = {'pin_memory': True}
train_dataloader = DataLoader(train_dataset, batch_size=2,
                                collate_fn=collate_fn, **kwargs)

Found cached dataset scrolls (/home/bulatov/.cache/huggingface/datasets/tau___scrolls/quality/1.0.0/672021d5d8e1edff998a6ea7a5bff35fdfd0ae243e7cf6a8c88a57a04afb46ac)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
gen = iter(train_dataloader)
batch = next(gen)

In [ ]:
batch.keys()

dict_keys(['input_ids', 'labels', 'labels_mask', 'attention_mask'])

In [ ]:
batch['input_ids'].shape

torch.Size([2, 128])

In [ ]:
batch['labels'].shape

torch.Size([2, 128])

In [ ]:
tokenizer.batch_decode(batch['input_ids'])

['Why is Si retirement so significant to the Space Exploration Team? \n\n (A) There aren’t enough working people in the world. They won’t be able to find a replacement.\n (B) As one of two remaining spacemen, it would likely mean the defunding and shut down of the Space Exploration Team.\n (C) Training new spacemen is costly and time consuming. They won’t have anyone else ready after him.\n (D) His retirement may inspire others to[GEN]Training new spacemen is costly and time consuming. They won’t have anyone else ready after him.',
 'What makes Gubelin an outlier in the present day?\n\n (A) He is much older than the rest of the population.\n (B) He refuses new operations that could improve his health.\n (C) His mind is still active, and he values hard work.\n (D) He still wears glasses and value objects like the gold watch given to Si.\n\n\nSPACEMAN ON A SPREE\n\n\n\n\n   BY MACK REYNOLDS\n\n\n\n\n   Illustrated by Nodel\n\n\n\n\n[GEN]He still wears glasses and value objects like the g

In [ ]:
tokenizer.batch_decode([c[i] for c, i in zip(batch['input_ids'], batch['labels_mask'])])

['[GEN]Training new spacemen is costly and time consuming. They won’t have anyone else ready after him.',
 '[GEN]He still wears glasses and value objects like the gold watch given to Si.']

In [ ]:
tokenizer.batch_decode([c[i] for c, i in zip(batch['input_ids'], batch['attention_mask'])])

['Why is Si retirement so significant to the Space Exploration Team? \n\n (A) There aren’t enough working people in the world. They won’t be able to find a replacement.\n (B) As one of two remaining spacemen, it would likely mean the defunding and shut down of the Space Exploration Team.\n (C) Training new spacemen is costly and time consuming. They won’t have anyone else ready after him.\n (D) His retirement may inspire others toTraining new spacemen is costly and time consuming. They won’t have anyone else ready after him.',
 'What makes Gubelin an outlier in the present day?\n\n (A) He is much older than the rest of the population.\n (B) He refuses new operations that could improve his health.\n (C) His mind is still active, and he values hard work.\n (D) He still wears glasses and value objects like the gold watch given to Si.\n\n\nSPACEMAN ON A SPREE\n\n\n\n\n   BY MACK REYNOLDS\n\n\n\n\n   Illustrated by Nodel\n\n\n\n\nHe still wears glasses and value objects like the gold watch 

In [ ]:
out = rmt(**batch)

In [ ]:
out.loss

tensor(60.2067, grad_fn=<NllLossBackward>)

In [ ]:
out.loss

tensor(61.4377, grad_fn=<NllLossBackward>)